In [1]:
%load_ext autoreload
%autoreload 2


In [ ]:
import torch



from src.utils.utils import lp_error
from src.utils.logger import Logging
from src.utils.colors import model_color

from src.nn.tanh2 import MLP2
from src.nn.tanh import MLP
from src.nn.bspline import KAN
from src.nn.kan2 import KAN2
from src.utils.utils import clear_gpu_memory
from src.data.IBM_data_loader import prepare_training_data, visualize_tensor_datasets
from src.data.IBM_data_loader import load_fluid_testing_dataset
from src.models.m1_physics2 import PINNTrainer
from src.utils.plot_losses import plot_M1_loss_history
from src.utils.fsi_visualization import (
    create_frames,
    create_animations_from_existing_frames,
)
from src.data.IBM_data_loader import load_training_dataset


In [3]:

CHECKPOINT_PATH = "./checkpoints"
logger = Logging(CHECKPOINT_PATH)
model_dirname = logger.get_output_dir()

logger.print(model_dirname)

INFO:src.utils.logger:./checkpoints/2025-08-17_00-29-12-363506


In [14]:


clear_gpu_memory()
config = {
    "dataset_type": "old",
    "training_selection_method": "Sobol",
    "input_dim": 3,  # (x, y, z, t)
    "hidden_dim": 300,
    "hidden_layers_dim": 3,
    "fluid_density": 1.0,
    "fluid_viscosity": 0.01,
    "num_epochs": 60000,
    "batch_size": 128,
    "learning_rate": 1e-3,
    "data_weight": 4.0,
    "physics_weight": 0.01,
    "boundary_weight": 2.0,
    "fsi_weight": 0.5,
    "initial_weight": 4.0,
    "checkpoint_dir": "./checkpoints",
    "resume": None,
    "print_every": 100,
    "save_every": 200,
    "fluid_sampling_ratio": 0.01,
    "interface_sampling_ratio": 0.07,
    "solid_sampling_ratio": 0.0,
    "left_sampling_ratio": 0.1,
    "right_sampling_ratio": 0.15,
    "bottom_sampling_ratio": 0.1,
    "top_sampling_ratio": 0.1,
    "initial_sampling_ratio": 0.1,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "solver": "mlp",
    "model": "m1",
}


GPU memory cleared!


In [15]:
logger.print("Config:")
for key, value in config.items():
    logger.print(f"{key}: {value}")

training_data_path = "./data/training_dataset/old"

training_data = load_training_dataset(training_data_path, device=config["device"])

if training_data is None:
    training_data = prepare_training_data(
        config["dataset_type"],
        fluid_sampling_ratio=config["fluid_sampling_ratio"],
        interface_sampling_ratio=config["interface_sampling_ratio"],
        solid_sampling_ratio=config["solid_sampling_ratio"],
        left_sampling_ratio=config["left_sampling_ratio"],
        right_sampling_ratio=config["right_sampling_ratio"],
        bottom_sampling_ratio=config["bottom_sampling_ratio"],
        top_sampling_ratio=config["top_sampling_ratio"],
        initial_sampling_ratio=config["initial_sampling_ratio"],
        training_selection_method=config["training_selection_method"],
        device=config["device"],
        save_dir=training_data_path,
    )


INFO:src.utils.logger:Config:
INFO:src.utils.logger:dataset_type: old
INFO:src.utils.logger:training_selection_method: Sobol
INFO:src.utils.logger:input_dim: 3
INFO:src.utils.logger:hidden_dim: 300
INFO:src.utils.logger:hidden_layers_dim: 3
INFO:src.utils.logger:fluid_density: 1.0
INFO:src.utils.logger:fluid_viscosity: 0.01
INFO:src.utils.logger:num_epochs: 60000
INFO:src.utils.logger:batch_size: 128


INFO:src.utils.logger:learning_rate: 0.001
INFO:src.utils.logger:data_weight: 4.0
INFO:src.utils.logger:physics_weight: 0.01
INFO:src.utils.logger:boundary_weight: 2.0
INFO:src.utils.logger:fsi_weight: 0.5
INFO:src.utils.logger:initial_weight: 4.0
INFO:src.utils.logger:checkpoint_dir: ./checkpoints
INFO:src.utils.logger:resume: None
INFO:src.utils.logger:print_every: 100
INFO:src.utils.logger:save_every: 200
INFO:src.utils.logger:fluid_sampling_ratio: 0.01
INFO:src.utils.logger:interface_sampling_ratio: 0.07
INFO:src.utils.logger:solid_sampling_ratio: 0.0
INFO:src.utils.logger:left_sampling_ratio: 0.1
INFO:src.utils.logger:right_sampling_ratio: 0.15
INFO:src.utils.logger:bottom_sampling_ratio: 0.1
INFO:src.utils.logger:top_sampling_ratio: 0.1
INFO:src.utils.logger:initial_sampling_ratio: 0.1
INFO:src.utils.logger:device: cuda
INFO:src.utils.logger:solver: mlp
INFO:src.utils.logger:model: m1


Loaded fluid_points tensor from ./data/training_dataset/old/fluid_points_tensor.pt with shape torch.Size([3986, 8])
Loaded right tensor from ./data/training_dataset/old/right_tensor.pt with shape torch.Size([1545, 8])
Loaded solid tensor from ./data/training_dataset/old/solid_tensor.pt with shape torch.Size([1452, 8])
Loaded initial tensor from ./data/training_dataset/old/initial_tensor.pt with shape torch.Size([1330, 8])
Loaded bottom tensor from ./data/training_dataset/old/bottom_tensor.pt with shape torch.Size([1030, 8])
Loaded fluid tensor from ./data/training_dataset/old/fluid_tensor.pt with shape torch.Size([10508, 8])
Loaded up tensor from ./data/training_dataset/old/up_tensor.pt with shape torch.Size([1030, 8])
Loaded interface tensor from ./data/training_dataset/old/interface_tensor.pt with shape torch.Size([10166, 11])
Loaded left tensor from ./data/training_dataset/old/left_tensor.pt with shape torch.Size([1030, 8])
Loaded training dataset from ./data/training_dataset/old su

In [16]:

visualize_tensor_datasets(training_data, save_dir=training_data_path)

fluid_network = (
    [config["input_dim"]] + [config["hidden_dim"]] * config["hidden_layers_dim"] + [3]
)
if config["solver"] == "mlp":
    fluid_model = MLP(network=fluid_network)
else:
    fluid_model = KAN2(network=fluid_network)

logger.print("Fluid model architecture:")
logger.print(fluid_model)
logger.print(
    f"Number of parameters: {sum(p.numel() for p in fluid_model.parameters())}"
)


INFO:src.utils.logger:Fluid model architecture:
INFO:src.utils.logger:MLP(
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=300, bias=True)
    (1-2): 2 x Linear(in_features=300, out_features=300, bias=True)
    (3): Linear(in_features=300, out_features=3, bias=True)
  )
)
INFO:src.utils.logger:Number of parameters: 182703


Saved tensor datasets scatter plot to ./data/training_dataset/old/tensor_datasets_scatter.png


In [17]:

trainer = PINNTrainer(
    fluid_model=fluid_model,
    training_data=training_data,
    learning_rate=config["learning_rate"],
    logger=logger,
    device=config["device"],
    fluid_density=config["fluid_density"],
    fluid_viscosity=config["fluid_viscosity"],
    print_every=config["print_every"],
    save_every=config["save_every"],
    solver=config["solver"],
    model=config["model"],
)


loss_history = trainer.train(
    num_epochs=config["num_epochs"],
    batch_size=config["batch_size"],
    data_weight=config["data_weight"],
    physics_weight=config["physics_weight"],
    boundary_weight=config["boundary_weight"],
    fsi_weight=config["fsi_weight"],
    initial_weight=config["initial_weight"],
)


INFO:src.utils.logger:Epoch 0/60000, Total: 2.0e+01, Data(F&S): 4.3e-01, Physics: 5.5e-03, Boundary: 4.2e+00, FSI: 1.5e+01, Initial: 1.5e-01, LR: 1.00e-03
INFO:src.utils.logger:Epoch 100/60000, Total: 2.4e+00, Data(F&S): 3.2e-01, Physics: 1.2e-03, Boundary: 1.9e+00, FSI: 1.1e-01, Initial: 1.2e-01, LR: 1.00e-03
INFO:src.utils.logger:Final losses:
INFO:src.utils.logger:Final left: 2.524e-01 |  Final right: 4.318e-01 |  Final bottom: 5.111e-02 |  Final up: 1.126e+00 |  Final fluid: 1.182e-03 |  Final interface: 9.330e-02 |  Final initial: 1.061e-01 |  Final total: 2.384e+00 | 
INFO:src.utils.logger:_save_checkpoint: Epoch 200 | Training checkpoint saved at ./checkpoints/2025-08-17_00-29-12-363506/model.pth
INFO:src.utils.logger:Epoch 200/60000, Total: 2.4e+00, Data(F&S): 3.2e-01, Physics: 1.2e-03, Boundary: 1.9e+00, FSI: 9.3e-02, Initial: 1.1e-01, LR: 1.00e-03
INFO:src.utils.logger:Epoch 300/60000, Total: 1.5e+00, Data(F&S): 1.7e-01, Physics: 2.8e-03, Boundary: 1.2e+00, FSI: 9.7e-02, Init

KeyboardInterrupt: 